In [1]:
import logging
import tensorflow as tf


GPU_FROM = 1
GPU_TO = 2    

visible_devices = tf.config.get_visible_devices('GPU')
logging.info(f"Num GPUs visible:{len(visible_devices)}")
tf.config.set_visible_devices(visible_devices[GPU_FROM:GPU_TO],'GPU')

visible_devices = tf.config.get_visible_devices('GPU')
logging.info(f"Num GPUs to be used: {len(visible_devices)}")

2023-02-20 18:21:04.417557: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-20 18:21:04.595704: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-20 18:21:04.635340: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-02-20 18:21:05.356414: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [2]:

from segmentation.model import SpaceSegmentationTransformer
from segmentation.model import LossWithVoids

gap = 3
len = 100

data = tf.random.stateless_binomial(shape=(10000, len), counts=1, probs=0.8, seed=[1997,1997]) + 1
train_frac = int(data.shape[0]*3/4)

train_ds, val_ds = tf.data.Dataset.from_tensor_slices(data[:train_frac]), tf.data.Dataset.from_tensor_slices(data[train_frac:])

def mapper(y):
    x = tf.strings.as_string(y+4)
    x = tf.strings.reduce_join(x, axis=-1)
    # offset_noise = 0
    offset_noise =  tf.random.uniform(shape=(), minval=0,maxval=3,dtype="int32")
    checkerboard = tf.where((tf.range(len) + offset_noise) % gap == 0, 2, 1)
    return (x, x), tf.cast(checkerboard, "float16")

train_ds = train_ds.map(mapper)
val_ds = val_ds.map(mapper)

print(train_ds.element_spec, val_ds.element_spec)
for x in train_ds.take(1):
    print(x)

((TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.string, name=None)), TensorSpec(shape=(100,), dtype=tf.float16, name=None)) ((TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.string, name=None)), TensorSpec(shape=(100,), dtype=tf.float16, name=None))
((<tf.Tensor: shape=(), dtype=string, numpy=b'6666656666666655666665566566666656666656666665566566566566666566656666665566666556666656666666666565'>, <tf.Tensor: shape=(), dtype=string, numpy=b'6666656666666655666665566566666656666656666665566566566566666566656666665566666556666656666666666565'>), <tf.Tensor: shape=(100,), dtype=float16, numpy=
array([1., 2., 1., 1., 2., 1., 1., 2., 1., 1., 2., 1., 1., 2., 1., 1., 2.,
       1., 1., 2., 1., 1., 2., 1., 1., 2., 1., 1., 2., 1., 1., 2., 1., 1.,
       2., 1., 1., 2., 1., 1., 2., 1., 1., 2., 1., 1., 2., 1., 1., 2., 1.,
       1., 2., 1., 1., 2., 1., 1., 2., 1., 1., 2., 1., 1., 2., 1., 1., 2.,
       1., 1., 2., 1., 1., 2., 1., 1.

2023-02-20 18:21:06.158750: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-20 18:21:06.839181: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22296 MB memory:  -> device: 1, name: GeForce RTX 3090, pci bus id: 0000:5e:00.0, compute capability: 8.6


In [3]:
class SparseAccuracyWithIgnore(tf.keras.metrics.SparseCategoricalAccuracy):
    def __init__(self, name="sparse_categorical_accuracy", dtype=None, ignore_token=None):
        self.ignore_token = ignore_token
        super().__init__(name, dtype)
    
    def update_state(self, y_true, y_pred, sample_weight=None):
        if self.ignore_token is not None:
            mask = tf.not_equal(y_true, tf.cast(self.ignore_token, y_true.dtype))
            y_true = tf.boolean_mask(y_true, tf.squeeze(mask))
            y_pred = tf.boolean_mask(y_pred, tf.squeeze(mask))
            # TODO: Confirm the masking semantics do as we expect
            # bit weird it doesn't return a ragged tensor
            # Also a bit weird it wasn't a problem before
        return super().update_state(y_true, y_pred)

m = SparseAccuracyWithIgnore(ignore_token=0)
m.update_state(
    tf.constant([[1], [2], [1], [1], [0], [0]]),
    tf.constant([
        [0, 0, 1], # 2 F
        [0, 0, 1], # 2 T
        [0, 1, 0], # 1 T
        [0, 1, 0], # 1 T
        [0, 1, 0], # 1 F - should be ignored
        [1, 0, 0]  # 0 T - should be ignored
    ])
)
m.ignore_token, m.result() # Wrong is 66% correct is 75%

(0, <tf.Tensor: shape=(), dtype=float32, numpy=0.75>)

In [4]:
tokenizer = tf.keras.layers.TextVectorization(
    output_sequence_length=100,
    standardize="lower_and_strip_punctuation",
    split="character",
    output_mode="int",
)

tokenizer.adapt(train_ds.take(10).map(lambda x,y: x[0]))
tokenizer.get_vocabulary()

['', '[UNK]', '6', '5']

In [5]:
tokenizer("5556665")

<tf.Tensor: shape=(100,), dtype=int64, numpy=
array([3, 3, 3, 2, 2, 2, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])>

In [6]:
model = SpaceSegmentationTransformer(
    num_layers=2,
    d_model=512,
    num_attention_heads=3,
    seq_len=100,
    dff=1028,
    input_tokenizer=tokenizer,
    dropout_rate=0.1,
    num_classes=3
)

In [7]:
from segmentation.metrics import SparseAccuracyWithIgnore
from segmentation.metrics import SparsePrecision
from segmentation.metrics import SparseRecall
from segmentation.metrics import SparseF1

model.compile(
    optimizer="adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=[
        SparseAccuracyWithIgnore(ignore_token=0.),
        SparsePrecision(class_id=2, name="space_precision"),
        SparseRecall(class_id=2, name="space_recall"),
        SparseF1(class_id=2, name="space_f1"),
        SparsePrecision(class_id=1, name="char_precision"),
        SparseRecall(class_id=1, name="char_recall"),
        SparseF1(class_id=1, name="char_f1"),
    ]
)

In [8]:
train_ds = train_ds.shuffle(100).batch(8)
val_ds = val_ds.batch(8)

callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_space_precision",
        patience=2,
        mode="max"
    )
]

model.fit(train_ds, validation_data=val_ds, epochs=100, callbacks=callbacks)

Epoch 1/100
  7/938 [..............................] - ETA: 20s - loss: 3.6451 - sparse_categorical_accuracy: 0.5045 - space_precision: 0.3367 - space_recall: 0.3713 - space_f1: 0.3532 - char_precision: 0.6659 - char_recall: 0.5712 - char_f1: 0.6149        

2023-02-20 18:21:12.888796: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


938/938 [==============================] - 29s 26ms/step - loss: 0.6674 - sparse_categorical_accuracy: 0.6625 - space_precision: 0.3344 - space_recall: 0.0112 - space_f1: 0.0218 - char_precision: 0.6666 - char_recall: 0.9883 - char_f1: 0.7962 - val_loss: 0.6382 - val_sparse_categorical_accuracy: 0.6667 - val_space_precision: 0.0000e+00 - val_space_recall: 0.0000e+00 - val_space_f1: nan - val_char_precision: 0.6667 - val_char_recall: 1.0000 - val_char_f1: 0.8000
Epoch 2/100
938/938 [==============================] - 23s 25ms/step - loss: 0.6372 - sparse_categorical_accuracy: 0.6666 - space_precision: 0.0000e+00 - space_recall: 0.0000e+00 - space_f1: nan - char_precision: 0.6666 - char_recall: 1.0000 - char_f1: 0.8000 - val_loss: 0.6366 - val_sparse_categorical_accuracy: 0.6666 - val_space_precision: 0.0000e+00 - val_space_recall: 0.0000e+00 - val_space_f1: nan - val_char_precision: 0.6666 - val_char_recall: 1.0000 - val_char_f1: 0.7999
Epoch 3/100
938/938 [==============================

In [9]:
model.evaluate(val_ds)

313/313 [==============================] - 4s 13ms/step - loss: 0.6365 - sparse_categorical_accuracy: 0.6668 - space_precision: 0.0000e+00 - space_recall: 0.0000e+00 - space_f1: nan - char_precision: 0.6668 - char_recall: 1.0000 - char_f1: 0.8001


[0.6364565491676331,
 0.6667519807815552,
 0.0,
 0.0,
 nan,
 0.6667519807815552,
 1.0,
 0.8000614047050476]

In [10]:
preds = model([("5555a", "5656555", "555555666666"),(None,None, None)])
preds, tf.argmax(preds, axis=-1)

(<tf.Tensor: shape=(3, 100, 3), dtype=float32, numpy=
 array([[[9.7392524e-07, 6.6626745e-01, 3.3373156e-01],
         [9.7377222e-07, 6.6628414e-01, 3.3371493e-01],
         [9.7383486e-07, 6.6628766e-01, 3.3371142e-01],
         [9.7399288e-07, 6.6629601e-01, 3.3370301e-01],
         [9.7395639e-07, 6.6626978e-01, 3.3372924e-01],
         [9.7337386e-07, 6.6631347e-01, 3.3368555e-01],
         [9.7337568e-07, 6.6631973e-01, 3.3367929e-01],
         [9.7318639e-07, 6.6631335e-01, 3.3368564e-01],
         [9.7321004e-07, 6.6631502e-01, 3.3368406e-01],
         [9.7328950e-07, 6.6632432e-01, 3.3367473e-01],
         [9.7342274e-07, 6.6631764e-01, 3.3368140e-01],
         [9.7362306e-07, 6.6630989e-01, 3.3368915e-01],
         [9.7375391e-07, 6.6631740e-01, 3.3368161e-01],
         [9.7347959e-07, 6.6630763e-01, 3.3369136e-01],
         [9.7338807e-07, 6.6629708e-01, 3.3370200e-01],
         [9.7319037e-07, 6.6630089e-01, 3.3369818e-01],
         [9.7338159e-07, 6.6631871e-01, 3.3368033e

In [11]:
mask = model.encoder.compute_mask(
    ("5555a", "5656555", "555555666666")
)
print(mask)

tf.Tensor(
[[ True  True  True  True  True False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False]
 [ True  True  True  True  True  True  True False False False False False
  False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False False False False False False
